# Calculations of the Effect Size (ES) for each microarray study 

###  Using Hedges' g value, an adjusted Cohen's d  value

$$  {Enrichment} = \bar{X_2}-\bar{X_1}$$

Let Group 1 be 12hSleeping whole brain DBA/2J strain (D2) Expression values and Group 2 be 12hSD whole brain DBA/2J strain (D2) Expression values 

(S mean - SD mean) **(Logged values, so minus gives ratio)** 

$$  {Pooled\ Standard\  Deviation} = \sqrt\frac{(n_1-1)S_1^2 +(n_2-1)S_2^2}{(n_2 +n_2) -2}  $$  

$$  {Cohen's\ d\ value} = \frac{Enrichment}{Pooled\ Standard\ Deviation} $$

$$  {Correction\ Factor (J\ Factor)} = 1- \frac{3}{4df-1} $$

$$  {Hedges'\ g\ value} = Cohen's\ d\ \text{x}\ J\ $$

$$  {Variance\ in\ d (V_d)} = \frac{n_1- +n_2}{n_1 n_2} + \frac{d^2}{2(n_1 +n_2)}  $$

$$  {Variance\ in\ g (V_g)} = J^2\  \text{x}\ V_d  $$

$$  {Standard\ Error\ in\ g (SE_g)} = \sqrt{V_g}  $$

## Setup working environment and import data

In [1]:
import pandas as pd # Dataframes and file IO
import numpy as np # numerical calculations
%cd /Users/Ella1/Desktop/data sets 430AV2


/Users/Ella1/Desktop/data sets 430AV2


In [2]:
prefix = '430AV2_ZT12_D2_'   # define a prefix to add to column names (mD2ing indexing easier later)

In [3]:
# import the data file to a data frame 'df'
df=pd.read_table('DATASET-GSE9442.txt', delimiter='\t',  index_col=0) #,nrows=500)  
df.shape

(45101, 87)

In [4]:
# remove probes that are know to cross-hybridise to more than one target
df =df[~df.index.str.contains('_x_|_s_')]    #   important reverse selector ~ 
df.shape

(40569, 87)

## Look at column names and then setup filters for grouping columns into S and SD groups

In [5]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites', 'Select Cellular Compartments',
       'Select Protein Classes', 'GSM239868_AK_S_ZT6.CEL',
       'GSM239869_AK_S_ZT6.CEL', 'GSM239870_AK_S_ZT6.CEL', 'avg-AK_S_ZT6',
       'log_fold-AK_S_ZT6_vs_AK_SD_ZT6', 'fold-AK_S_ZT6_vs_AK_SD_ZT6',
       'rawp-AK_S_ZT6_vs_AK_SD_ZT6', 'adjp-AK_S_ZT6_vs_AK_SD_ZT6',
       'GSM239871_AK_S_ZT12.CEL', 'GSM239872_AK_S_ZT12.CEL',
       'GSM239873_AK_S_ZT12.CEL', 'avg-AK_S_ZT12',
       'log_fold-AK_S_ZT12_vs_AK_SD_ZT12', 'fold-AK_S_ZT12_vs_AK_SD_ZT12',
       'rawp-AK_S_ZT12_vs_AK_SD_ZT12', 'adjp-AK_S_ZT12_vs_AK_SD_ZT12',
       'GSM239880_AK_SD_ZT6.CEL', 'GSM239881_AK_SD_ZT6.CEL',
       'GSM239882_AK_SD_ZT6.CEL', 'avg-AK_SD_ZT6', 'GSM239883_AK_SD_ZT12.CEL',
       'GSM239884_AK_SD_ZT12.CEL', 'GSM239885_AK_SD_ZT12.CEL',
       'avg-AK_SD_ZT12', 'GSM239891_B6_S_ZT6.CEL',
  

In [6]:
# define regular expressions for sleep (S) and sleep dep (SD) filters 
s_filt ='D2_S_ZT12.CEL'
sd_filt ='D2_SD_ZT12.CEL'

In [7]:
df_s=df.filter(regex= s_filt)
df_s.head()

,GSM239918_D2_S_ZT12.CEL,GSM239919_D2_S_ZT12.CEL,GSM239920_D2_S_ZT12.CEL
Probesets,,,
1427138_at,6.11537,6.25896,5.97517
1425600_a_at,7.50926,7.66500,7.42895
1457168_at,4.79932,4.65590,4.59068
1450135_at,5.20164,5.16381,4.91162
1424014_at,8.12506,8.07303,8.19931


In [8]:
df_sd=df.filter(regex= sd_filt)
df_sd.head()

,GSM239930_D2_SD_ZT12.CEL,GSM239931_D2_SD_ZT12.CEL,GSM239932_D2_SD_ZT12.CEL
Probesets,,,
1427138_at,6.06469,6.18215,6.18784
1425600_a_at,7.52460,7.28965,7.76306
1457168_at,4.44227,4.63462,4.49105
1450135_at,5.37315,5.15923,5.18314
1424014_at,8.20743,8.27671,8.21147


## Calculations 

In [9]:
# Enrichment

df[prefix+'Enrich'] = df.filter(regex=sd_filt).mean(axis=1) - df.filter(regex=s_filt).mean(axis=1)

In [10]:
df[prefix+'Enrich'].head()

Probesets
1427138_at      0.028393
1425600_a_at   -0.008633
1457168_at     -0.159320
1450135_at      0.146150
1424014_at      0.099403
Name: 430AV2_ZT12_D2_Enrich, dtype: float64

In [11]:
# Calculating Pooled StDev
Scount = df.filter(regex=s_filt).count(axis=1)
SDcount = df.filter(regex=sd_filt).count(axis=1)

StdevS = (Scount-1) * df.filter(regex=s_filt).var(axis=1)
StdevSD = (SDcount-1) * df.filter(regex=sd_filt).var(axis=1)

df[prefix+'poolStDev'] = np.sqrt((StdevS+StdevSD)/(Scount+ SDcount-2))

In [12]:
# Calculating Cohen's d
df[prefix+'Cohens_d'] = df[prefix+'Enrich'] / df[prefix+'poolStDev']

In [13]:
#df[prefix+'poolStDev'].head()
df[prefix+'Cohens_d'] .head()

Probesets
1427138_at      0.254122
1425600_a_at   -0.046005
1457168_at     -1.540541
1450135_at      1.052057
1424014_at      1.888717
Name: 430AV2_ZT12_D2_Cohens_d, dtype: float64

In [14]:
# Calculating J value (Correction factor)

df[prefix+'J'] = 1-(3/(4*(Scount+SDcount-1)))                              


In [15]:
# Calculating Hedge's g

df[prefix+'Hedges_g'] = df[prefix+'Cohens_d'] * df[prefix+'J']

In [16]:
#df[prefix+'J'].head()
df[prefix+'Hedges_g'] .head()

Probesets
1427138_at      0.216004
1425600_a_at   -0.039104
1457168_at     -1.309460
1450135_at      0.894248
1424014_at      1.605409
Name: 430AV2_ZT12_D2_Hedges_g, dtype: float64

In [17]:
# Calculating Var_d
Scount = df.filter(regex=s_filt).count(axis=1)
SDcount = df.filter(regex=sd_filt).count(axis=1)

Ftop1 = Scount + SDcount
Ftop2 = Scount * SDcount
Fbottom1 = np.square(df[prefix+'Cohens_d']) 
Fbottom2 =  2*(Scount + SDcount)


df[prefix+'Var_d'] = (Ftop1/Ftop2) + (Fbottom1 /Fbottom2)

In [18]:
#check output
df[prefix+'Var_d'].head()

Probesets
1427138_at      0.672048
1425600_a_at    0.666843
1457168_at      0.864439
1450135_at      0.758902
1424014_at      0.963938
Name: 430AV2_ZT12_D2_Var_d, dtype: float64

In [19]:
df[prefix+'Var_g'] = df[prefix+'Var_d'] * np.square(df[prefix+'J'])

In [20]:
# Calculating SEg
df[prefix+'SEg'] = np.sqrt(df[prefix+'Var_g'])

In [21]:
df.sort_values(by= '430AV2_ZT12_D2_Hedges_g', ascending=False, inplace=True)
df

,Symbol,Definition,Ensembl_id,Entrez_id,Unigene_id,GO-Process,GO-Function,GO-Component,Pathway_info,Putative microRNA binding sites,...,ANOVA-adjp,largest fold,430AV2_ZT12_D2_Enrich,430AV2_ZT12_D2_poolStDev,430AV2_ZT12_D2_Cohens_d,430AV2_ZT12_D2_J,430AV2_ZT12_D2_Hedges_g,430AV2_ZT12_D2_Var_d,430AV2_ZT12_D2_Var_g,430AV2_ZT12_D2_SEg
Probesets,,,,,,,,,,,,,,,,,,,,,
1428112_at,Armet,"arginine-rich, mutated in early stage tumors",ENSMUSG00000032575,74840,NaN,NaN,growth factor activity,extracellular region,NaN,"mmu-let-7f(RNAhybrid|miRanda), mmu-miR-106a(mi...",...,1.042066e-12,0.995037,0.925650,0.029378,31.508166,0.85,26.781941,83.397042,60.254363,7.762368
1424033_at,Sfrs7,"splicing factor, arginine/serine-rich 7",ENSMUSG00000024097,225027,NaN,RNA splicing // mRNA processing,nucleic acid binding // RNA binding // zinc io...,nucleus,mRNA processing:WP310(WikiPathways),"mmu-miR-1(pictar), mmu-miR-101a(miRanda), mmu-...",...,2.531613e-08,0.462230,0.293207,0.009624,30.467091,0.85,25.897027,78.020302,56.369668,7.507974
1416455_a_at,Cryab,"crystallin, alpha B",ENSMUSG00000032060,12955,NaN,camera-type eye development // response to hea...,structural constituent of eye lens,insoluble fraction // soluble fraction // cont...,NaN,"mmu-miR-101b(RNAhybrid|miRanda), mmu-miR-103(R...",...,6.945601e-08,0.843033,0.513307,0.017861,28.738856,0.85,24.428027,69.493485,50.209043,7.085834
1445617_at,NaN,NaN,ENSMUSG00000098020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.358618e-11,1.523407,0.607033,0.024295,24.985645,0.85,21.237798,52.690204,38.068672,6.169982
1416064_a_at,Hspa5,heat shock protein 5,ENSMUSG00000026864,14828,NaN,response to stress // ER overload response,protein binding // nucleotide binding // ATP b...,endoplasmic reticulum lumen // melanosome // e...,MAPK signaling pathway:WP493(WikiPathways),"mmu-miR-1192(TargetScan|miRanda), mmu-miR-1193...",...,2.958798e-12,1.152830,1.057613,0.047759,22.144775,0.85,18.823059,41.532588,30.007295,5.477891
1428620_at,Ensa,endosulfine alpha,ENSMUSG00000038619,56205,NaN,regulation of insulin secretion,potassium channel inhibitor activity // recept...,cytoplasm,NaN,"mmu-miR-1(miRanda), mmu-miR-103(miRanda), mmu-...",...,1.867645e-04,0.353160,0.223727,0.010668,20.971399,0.85,17.825690,37.316633,26.961267,5.192424
1452247_at,Fxr1,"fragile X mental retardation gene 1, autosomal...",ENSMUSG00000027680,14359,NaN,skeletal muscle development // multicellular o...,RNA binding // protein binding,cytoplasm // costamere,GenMAPP-mRNA_processing_binding_Reactome // mR...,"mmu-miR-1(miRanda), mmu-miR-100(RNAhybrid|miRa...",...,7.432957e-05,0.572753,0.418270,0.021560,19.400683,0.85,16.490581,32.032209,23.143271,4.810745
1417258_at,Cct5,chaperonin subunit 5 (epsilon),ENSMUSG00000022234,12465,NaN,cellular protein metabolic process // protein ...,protein binding // nucleotide binding // ATP b...,cytoplasm // chaperonin-containing T-complex,NaN,"mmu-miR-103(RNAhybrid), mmu-miR-107(RNAhybrid)...",...,1.728744e-02,0.282617,0.164087,0.008569,19.149415,0.85,16.277003,31.225007,22.560068,4.749744
1436387_at,C330006P03Rik,homer homolog 1 (Drosophila) [Source:MGI Symbo...,ENSMUSG00000007617,320588,NaN,NaN,NaN,NaN,NaN,"mmu-let-7a(RNAhybrid|miRanda), mmu-let-7b(RNAh...",...,1.535314e-16,2.583807,2.385450,0.124746,19.122517,0.85,16.254140,31.139222,22.498088,4.743215


In [22]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites', 'Select Cellular Compartments',
       'Select Protein Classes', 'GSM239868_AK_S_ZT6.CEL',
       'GSM239869_AK_S_ZT6.CEL', 'GSM239870_AK_S_ZT6.CEL', 'avg-AK_S_ZT6',
       'log_fold-AK_S_ZT6_vs_AK_SD_ZT6', 'fold-AK_S_ZT6_vs_AK_SD_ZT6',
       'rawp-AK_S_ZT6_vs_AK_SD_ZT6', 'adjp-AK_S_ZT6_vs_AK_SD_ZT6',
       'GSM239871_AK_S_ZT12.CEL', 'GSM239872_AK_S_ZT12.CEL',
       'GSM239873_AK_S_ZT12.CEL', 'avg-AK_S_ZT12',
       'log_fold-AK_S_ZT12_vs_AK_SD_ZT12', 'fold-AK_S_ZT12_vs_AK_SD_ZT12',
       'rawp-AK_S_ZT12_vs_AK_SD_ZT12', 'adjp-AK_S_ZT12_vs_AK_SD_ZT12',
       'GSM239880_AK_SD_ZT6.CEL', 'GSM239881_AK_SD_ZT6.CEL',
       'GSM239882_AK_SD_ZT6.CEL', 'avg-AK_SD_ZT6', 'GSM239883_AK_SD_ZT12.CEL',
       'GSM239884_AK_SD_ZT12.CEL', 'GSM239885_AK_SD_ZT12.CEL',
       'avg-AK_SD_ZT12', 'GSM239891_B6_S_ZT6.CEL',
  

### Import key file from BioMart and index probesets to MGI gene symbols

In [23]:
dfX=pd.read_table('../FHS project/Sleep notebook Copy/BioMart_Ensmbl_index/mart_export72_430v2430Av2.txt',index_col=[3])
 
dfX.pop('Affy mouse430 2 probeset') # remove 430V2 probeset info (not needed for 430AV2 indexing)
dfX.head(5)

,Ensembl Gene ID,Description,MGI symbol
Affy mouse430a 2 probeset,,,
1417126_a_at,ENSMUSG00000039221,ribosomal protein L22 like 1 [Source:MGI Symbo...,Rpl22l1
NaN,ENSMUSG00000095611,predicted gene 10597 [Source:MGI Symbol;Acc:MG...,Gm10597
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
NaN,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1


In [24]:
df_Join = df.join(dfX, how='left', sort=True)
df_FINAL1 = df_Join.groupby('MGI symbol').mean()
df_FINAL1[df_FINAL1.index.duplicated()==True]   # checking that no duplicate entries exist in the dataframe

,GSM239868_AK_S_ZT6.CEL,GSM239869_AK_S_ZT6.CEL,GSM239870_AK_S_ZT6.CEL,avg-AK_S_ZT6,log_fold-AK_S_ZT6_vs_AK_SD_ZT6,fold-AK_S_ZT6_vs_AK_SD_ZT6,rawp-AK_S_ZT6_vs_AK_SD_ZT6,adjp-AK_S_ZT6_vs_AK_SD_ZT6,GSM239871_AK_S_ZT12.CEL,GSM239872_AK_S_ZT12.CEL,...,ANOVA-adjp,largest fold,430AV2_ZT12_D2_Enrich,430AV2_ZT12_D2_poolStDev,430AV2_ZT12_D2_Cohens_d,430AV2_ZT12_D2_J,430AV2_ZT12_D2_Hedges_g,430AV2_ZT12_D2_Var_d,430AV2_ZT12_D2_Var_g,430AV2_ZT12_D2_SEg
MGI symbol,,,,,,,,,,,,,,,,,,,,,


### Columns from the list above can then easily be picked to produce files for use later. Examples below given:
 #### df3 = average S and SD expression for the platform and the log-fold changes
 #### df4 = Hedges g  values and associated variance for Meta-analysis (after indexing)

In [25]:
# df3 = df_FINAL1.loc[:,[u'avg-SD', u'avg-S', u'log_fold-S_vs_SD']]
# df3.columns =[prefix+'avg-SD', prefix+'avg-S', prefix+'log_fold-S_vs_SD']
# df3.to_csv('input_files/430AV2_SymbolExpression_forIndex.csv')

In [26]:
df4 = df_FINAL1.loc[:,[u'430AV2_ZT12_D2_Enrich',u'430AV2_ZT12_D2_Hedges_g', u'430AV2_ZT12_D2_Var_g', u'430AV2_ZT12_D2_SEg']]
df4.to_csv('../FHS project/Sleep notebook Copy/IPython_notebooks/input_files/430AV2_ZT12_D2_SymbolforIndexHedges.csv')

In [27]:
df4.head(10)  # check final ouput

,430AV2_ZT12_D2_Enrich,430AV2_ZT12_D2_Hedges_g,430AV2_ZT12_D2_Var_g,430AV2_ZT12_D2_SEg
MGI symbol,,,,
0610005C13Rik,-0.024660,-0.228121,0.486003,0.697139
0610008F07Rik,0.026037,0.317124,0.490047,0.700034
0610009B22Rik,0.085450,1.455174,0.658128,0.811251
0610009D07Rik,-0.102432,-1.022533,0.584713,0.763107
0610009O20Rik,-0.009613,-0.156791,0.483715,0.695496
0610010K14Rik,-0.135040,-1.007800,0.566305,0.752532
0610012G03Rik,0.115674,1.905911,1.325290,1.041554
0610031J06Rik,0.048470,0.695318,0.521956,0.722465
0610037L13Rik,-0.150977,-1.874220,0.774392,0.879995
